In [1]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

# Sudoku

The sudoku we want to solve is shown below:
    <img src="sudoku.png">
It can be represented as a list of lists.

In [2]:
def create_puzzle():
    return [ ["*",  3 ,  9 , "*", "*", "*", "*", "*",  7 ], 
             ["*", "*", "*",  7 , "*", "*",  4 ,  9 ,  2 ],
             ["*", "*", "*", "*",  6 ,  5 , "*",  8 ,  3 ],
             ["*", "*", "*",  6 , "*",  3 ,  2 ,  7 , "*"],
             ["*", "*", "*", "*",  4 , "*",  8 , "*", "*"],
             [ 5 ,  6 , "*", "*", "*", "*", "*", "*", "*"],
             ["*", "*",  5 ,  2 , "*",  9 , "*", "*",  1 ],
             ["*",  2 ,  1 , "*", "*", "*", "*",  4 , "*"],
             [ 7 , "*", "*", "*", "*", "*",  5 , "*", "*"]
           ]

The function `allDifferent` takes a list `L` as its argument.  This list contains both variables and numbers.
It returns a set of formulas expressing that all variables from `L` take different values.

In [3]:
def allDifferent(Variables):
    return { f'{x} != {y}' for x in Variables 
                           for y in Variables
                           if  x < y
           }

In [4]:
def constraints_from_puzzle(Puzzle):
    return { f'V{row+1}{col+1} == {Puzzle[row][col]}' for row in range(9)
                                                      for col in range(9)
                                                      if  Puzzle[row][col] != '*'
           }

In [5]:
constraints_from_puzzle(create_puzzle())

{'V12 == 3',
 'V13 == 9',
 'V19 == 7',
 'V24 == 7',
 'V27 == 4',
 'V28 == 9',
 'V29 == 2',
 'V35 == 6',
 'V36 == 5',
 'V38 == 8',
 'V39 == 3',
 'V44 == 6',
 'V46 == 3',
 'V47 == 2',
 'V48 == 7',
 'V55 == 4',
 'V57 == 8',
 'V61 == 5',
 'V62 == 6',
 'V73 == 5',
 'V74 == 2',
 'V76 == 9',
 'V79 == 1',
 'V82 == 2',
 'V83 == 1',
 'V88 == 4',
 'V91 == 7',
 'V97 == 5'}

The function `sudoku_csp` returns a CSP that encodes the given sudoku as a CSP.

In [6]:
def sudoku_csp():
    Puzzle = create_puzzle()
    L           = range(1, 9+1)
    Variables   = { f'V{row}{col}' for row in L
                                   for col in L
                  }
    Values      = set(L)
    Constraints = constraints_from_puzzle(Puzzle)
    # all entries in a row are unique
    for row in L:
        Constraints |= allDifferent({ f'V{row}{col}' for col in L })
    # all entries in a column are unique
    for col in L:
        Constraints |= allDifferent({ f'V{row}{col}' for row in L })
    # all entries in a square are unique    
    for r in range(3):
        for c in range(3):
            S = { f'V{r*3+row}{c*3+col}' for row in range(1, 3+1)
                                         for col in range(1, 3+1) 
                }
            Constraints |= allDifferent(S)
    return Variables, Values, Constraints

In [7]:
sudoku_csp()

({'V11',
  'V12',
  'V13',
  'V14',
  'V15',
  'V16',
  'V17',
  'V18',
  'V19',
  'V21',
  'V22',
  'V23',
  'V24',
  'V25',
  'V26',
  'V27',
  'V28',
  'V29',
  'V31',
  'V32',
  'V33',
  'V34',
  'V35',
  'V36',
  'V37',
  'V38',
  'V39',
  'V41',
  'V42',
  'V43',
  'V44',
  'V45',
  'V46',
  'V47',
  'V48',
  'V49',
  'V51',
  'V52',
  'V53',
  'V54',
  'V55',
  'V56',
  'V57',
  'V58',
  'V59',
  'V61',
  'V62',
  'V63',
  'V64',
  'V65',
  'V66',
  'V67',
  'V68',
  'V69',
  'V71',
  'V72',
  'V73',
  'V74',
  'V75',
  'V76',
  'V77',
  'V78',
  'V79',
  'V81',
  'V82',
  'V83',
  'V84',
  'V85',
  'V86',
  'V87',
  'V88',
  'V89',
  'V91',
  'V92',
  'V93',
  'V94',
  'V95',
  'V96',
  'V97',
  'V98',
  'V99'},
 {1, 2, 3, 4, 5, 6, 7, 8, 9},
 {'V11 != V12',
  'V11 != V13',
  'V11 != V14',
  'V11 != V15',
  'V11 != V16',
  'V11 != V17',
  'V11 != V18',
  'V11 != V19',
  'V11 != V21',
  'V11 != V22',
  'V11 != V23',
  'V11 != V31',
  'V11 != V32',
  'V11 != V33',
  'V11 != V41',


Given a sudoku `Puzzle` and a `Solution` for this puzzle, the function `find_alternative` computes a CSP
that is solvable iff the puzzle has a second solution that is different from `Solution`.

In [8]:
def find_alternative(csp, Solution):
    Variables, Values, Constraints = csp
    formula = ' or '.join({ f'{var} != {Solution[var]}' for var in Variables })
    return Variables, Values, Constraints | { formula }

## Graphical Representation

The following line needs to be executed once to install the package `problem_visuals`.

In [9]:
!pip install git+https://github.com/reclinarka/problem_visuals

  Cloning https://github.com/reclinarka/problem_visuals to /private/var/folders/j0/9pg05lc57pd41f8b7d_b7sv40000gq/T/pip-req-build-1_p8yf6w
  Running command git clone --filter=blob:none --quiet https://github.com/reclinarka/problem_visuals /private/var/folders/j0/9pg05lc57pd41f8b7d_b7sv40000gq/T/pip-req-build-1_p8yf6w
  Resolved https://github.com/reclinarka/problem_visuals to commit 686d00067dd5f1eee5f227d2967368eba74f11b3
  Preparing metadata (setup.py) ... done
  Created wheel for problem-visuals: filename=problem_visuals-0.6.1-py3-none-any.whl size=15800 sha256=55fcafedf48b46e554f2f21466c5b4e72a7fd93d94ec103092f39084ed77f7bc
  Stored in directory: /private/var/folders/j0/9pg05lc57pd41f8b7d_b7sv40000gq/T/pip-ephem-wheel-cache-o47g6ukm/wheels/94/73/6d/92b7b9904c480ab6a5e8d7acd2354ba7bd62644913a2616f10
Successfully built problem-visuals


In [10]:
from problem_visuals.games.sudoku.grid import Grid

The function `show_solution` prints the solution.

In [11]:
def show_solution(Solution, width='50%'):
    Sudoku = create_puzzle()
    Assignment = Solution.copy()
    for row in range(9):
        for col in range(9):
            if Sudoku[row][col] != '*':
                del Assignment[f'V{row+1}{col+1}']
    return Grid(state=Sudoku, assigned=Assignment, html_width=width)